In [1]:
%load_ext autoreload
%autoreload 2

import os
import pathlib
cwd = pathlib.Path.cwd()
if cwd.name != 'ryn':
    print('changing directory')
    os.chdir(cwd.parent)

import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

changing directory


In [2]:
from ryn.graphs import split
from ryn.graphs import loader

g = loader.load_graphs_from_uri('oke.fb15k237-trainvalidtest')[0]
print(f'loaded {g.str_stats}')

rels = split.Relation.from_graph(g)
rels.sort(key=lambda rel: rel.ratio)
print(f'retrieved {len(rels)} relations')

loaded ryn graph: oke.fb15k237-trainvalidtest
  nodes: 14541
  edges: 310116 (237 types)
  degree:
    mean 42.65
    median 26

retrieved 237 relations


In [ ]:
from tabulate import tabulate

rows = [(r.r, r.ratio, len(r.hs), len(r.ts), r.name) for r in rels]

N = 5

print(f'first {N}')
print(tabulate(rows[:N]))

print(f'mid {N}')
m = len(rows) / 2
print(tabulate(rows[int(m-N/2):int(m+N/2)]))

print(f'last {N}')
print(tabulate(rows[-N:]))

In [ ]:
import ryn
from ryn.common import plotter

from functools import partial

path = ryn.ENV.CACHE_DIR / 'notes.graph.split' / f'{g.name}.ratio'
prob = partial(split._prob, a=cfg.prob_a, o=cfg.prob_o, s=cfg.prob_s)

plt = plotter.Plotter(title=f'Ratio Distribution {g.name}', xlabel='Relation', ylabel='Ratio', fname=str(path))
plt.ax.scatter(range(len(rels)), [r.ratio for r in rels], color=plotter.CLR[0], s=1)
plt.ax.scatter(range(len(rels)), [prob(x) for x in range(len(rels))], color=plotter.CLR[1], s=1)

print()
plt.plot()
print()

print(tabulate(rows[100:105]))
print([f'{prob(x):.2f}' for x in range(80,90)])

In [13]:
soft_cfg = split.SoftConfig(
    owcw_split=.7, trainvalid_split=.7, pathname='soft',
    prob_a=3, prob_o=len(rels) / 2, prob_s=1/20)

_p = int(soft_cfg.owcw_split * 100)
print(f'creating a soft split of {_p}% ow and {100 - _p}% cw data')

seeds = [30061990, 8051991, 25031990, 2041992]
seeds = [30061990, ]
split.create_soft(g, soft_cfg, rels, seeds)
print('done')

creating a greedy split of 70% ow and 30% cw data


done


In [11]:
print(f'{len(g.source.triples)=}')

rtriples = [g.find(edges={r}) for r in g.source.rels.keys()]
print(f'{sum(map(len, rtriples))=}')


len(g.source.triples)=310116
sum(map(len, rtriples))=310116
